In [ ]:
%matplotlib inline

In [ ]:
"""
Script python pour ouvrir les fichiers de traces de clavier

"""
import matplotlib.pyplot as plt
import numpy as np
import time

def read_int(f):
    ba = bytearray(4)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.int32)
    return prm[0]

def read_double(f):
    ba = bytearray(8)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.double)
    return prm[0]

def read_double_tab(f, n):
    ba = bytearray(8*n)
    nr = f.readinto(ba)
    if nr != len(ba):
        return []
    else:
        prm = np.frombuffer(ba, dtype=np.double)
        return prm

def get_pics_from_file(filename):
    # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
    print("Ouverture du fichier de pics "+filename)
    f_pic = open(filename, "rb")
    info = dict()
    info["nb_pics"] = read_int(f_pic)
    print("Nb pics par trame: " + str(info["nb_pics"]))
    info["freq_sampling_khz"] = read_double(f_pic)
    print("Frequence d'echantillonnage: " + str(info["freq_sampling_khz"]) + " kHz")
    info["freq_trame_hz"] = read_double(f_pic)
    print("Frequence trame: " + str(info["freq_trame_hz"]) + " Hz")
    info["freq_pic_khz"] = read_double(f_pic)
    print("Frequence pic: " + str(info["freq_pic_khz"]) + " kHz")
    info["norm_fact"] = read_double(f_pic)
    print("Facteur de normalisation: " + str(info["norm_fact"]))
    tab_pics = []
    pics = read_double_tab(f_pic, info["nb_pics"])
    nb_trames = 1
    while len(pics) > 0:
        nb_trames = nb_trames+1
        tab_pics.append(pics)
        pics = read_double_tab(f_pic, info["nb_pics"])
    print("Nb trames: " + str(nb_trames))
    f_pic.close()
    return tab_pics, info
  
def get_pics_from_file2(filename):
  # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
  f_pic = open(filename, "rb")
  info = dict()
  info["nb_pics"] = read_int(f_pic)
  info["freq_sampling_khz"] = read_double(f_pic)
  info["freq_trame_hz"] = read_double(f_pic)
  info["freq_pic_khz"] = read_double(f_pic)
  info["norm_fact"] = read_double(f_pic)
  tab_pics = []
  pics = read_double_tab(f_pic, info["nb_pics"])
  nb_trames = 1
  while len(pics) > 0:
    nb_trames = nb_trames+1
    tab_pics.append(pics)
    pics = read_double_tab(f_pic, info["nb_pics"])
  f_pic.close()
  return tab_pics, info

In [ ]:
# Creating lists for data
# frames : all frames
# letters : letter correspondig to frames
import os
import pandas as pd

nb = 0
frames = []
letters = []
labl = []
for file in os.listdir("data/"):
    df = pd.DataFrame(columns=["trame", "label"])
    pics, info = get_pics_from_file2("data/" + file)
    label = file.split('_')[1]
    label = label.split(".")[0]
    if (label == "LOGINMDP"):
        continue
    labl.append(label)
    frames.extend(pics)
    letters.extend([label] * len(pics))

print(labl)

['0', '1', '2', '4', '3', '5', '6', '7', '8', '9', 'B', 'C', 'A', 'CTRL', 'E', 'D', 'ENTER', 'F', 'G', 'I', 'L', 'J', 'H', 'M', 'NOKEY', 'N', 'K', 'Y', 'U', 'Q', 'Z', 'SUPPR', 'V', 'T', 'O', 'R', 'SPACE', 'P', 'W', 'X', 'SHIFT', 'S']


In [ ]:
# Splitting the data for train/test/validation sets.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(frames, letters, train_size=0.7, random_state=42, stratify=letters)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=42, stratify=y_test)
print(f"Train labels:\n", len(y_train))
print(f"Test labels:\n", len(y_test))
print(f"Test labels:\n", len(y_val))

Train labels:
 246128
Test labels:
 52742
Test labels:
 52742


In [ ]:
X_train = np.array(X_train, dtype=np.double)
X_test = np.array(X_test, dtype=np.double)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
X_val = np.array(X_val, dtype=np.double)
y_val = np.array(y_val)
labl = np.asarray(labl)

In [ ]:
# KNN scikit learn version
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import functools
import numpy as np

#train
knn = KNeighborsClassifier(n_neighbors=1, n_jobs= -1)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
import sklearn

y_predict = knn.predict(X_val)
sklearn.metrics.accuracy_score(y_val, y_predict)

0.5115467748663305

In [ ]:
login_mdp, info = get_pics_from_file3('data/pics_LOGINMDP.bin')
all_letters = knn.predict(login_mdp)

In [ ]:
# Function that takes the majority letter in an array of letters.
def mean_letters(start, end, letters):
    d =  {}
    for i in range(start, end):
        if letters[i] not in d:
            d[letters[i]] = 0
        d[letters[i]] += 1
    maxi = letters[start]
    for letter in d:
        if d[letter] > d[maxi]:
            maxi = letter
    return maxi

In [ ]:
mean_letters(0, 5, ["I", "I", "E", "E", "E", "E", "E", "E", "E", "E", "E", "E"])

'E'

In [ ]:
# Creates blocs of letters based on repition patterns and extract mean signal of each bloc

login_mdp = np.array(login_mdp)
blocs = []
mean_signal_blocs = []
bloc_index = 0
in_bloc = False
min_bloc_size = 7

for i in range(min_bloc_size, len(all_letters)):
    mean_bloc = mean_letters(i-min_bloc_size, i, all_letters)
    if mean_bloc != 'NOKEY' and not in_bloc:
        bloc_index = i - min_bloc_size
        in_bloc = True
    elif in_bloc and mean_bloc == 'NOKEY':
        in_bloc = False
        blocs.append(all_letters[bloc_index:i-min_bloc_size])
        mean_signal_blocs.append(login_mdp[bloc_index:i-min_bloc_size].mean(axis=0))
print(blocs)

[array(['NOKEY', 'NOKEY', 'NOKEY', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'NOKEY', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', '1', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'W', 'CTRL', 'CTRL', 'CTRL', 'NOKEY', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
       'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL', 'CTRL',
      

In [ ]:
# Extract the main letter of each bloc to find the password.

message = []
for bloc in blocs:
    message.append(mean_letters(0, len(bloc), bloc))
print(message)

['CTRL', 'SHIFT', 'U', 'A', 'W', 'C', 'K', 'A', 'T', 'O', 'N', '3', '2', '0', '2', '3', 'ENTER']


In [ ]:
# knn on the mean signal of each block doesn't give good results ... (we get prediction further from "HACKATON", which is the expected pattern)
knn.predict(mean_signal_blocs)

array(['CTRL', 'SHIFT', 'U', 'A', 'A', 'E', 'K', 'A', 'Y', 'L', 'Y', '3',
       '2', 'SUPPR', '4', '4', 'ENTER'], dtype='<U5')